In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv('data.csv')

# Handle missing values
df.fillna(method='ffill', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
df['normalized_value'] = scaler.fit_transform(df['value'].values.reshape(-1, 1))

# Split into training and test sets (e.g., 80% train, 20% test)
train_size = int(len(df) * 0.8)
train_data, test_data = df.iloc[:train_size], df.iloc[train_size:]

# Optionally, perform cross-validation on the training set

# Further processing as needed for model input

# Example output
print("Training set shape:", train_data.shape)
print("Test set shape:", test_data.shape)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load and preprocess the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Define function to create dataset with input and output sequences
def create_dataset(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)

# Set time steps and split into train and test sets
time_steps = 30
X, y = create_dataset(data_scaled, time_steps)
X_train, X_test = X[:int(X.shape[0]*0.8)], X[int(X.shape[0]*0.8):]
y_train, y_test = y[:int(y.shape[0]*0.8)], y[int(y.shape[0]*0.8):]

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Plot training loss and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Invert predictions to original scale
train_predictions = scaler.inverse_transform(train_predictions)
y_train = scaler.inverse_transform([y_train])
test_predictions = scaler.inverse_transform(test_predictions)
y_test = scaler.inverse_transform([y_test])

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train[0], train_predictions[:,0]))
test_rmse = np.sqrt(mean_squared_error(y_test[0], test_predictions[:,0]))
print('Train RMSE:', train_rmse)
print('Test RMSE:', test_rmse)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')

# Preprocessing
# Check for missing values
missing_values = data.isnull().sum()
print("Missing Values:\n", missing_values)

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Splitting into training and test sets
train_size = int(len(data_scaled) * 0.8)  # 80% for training
test_size = len(data_scaled) - train_size
train, test = data_scaled[0:train_size,:], data_scaled[train_size:len(data_scaled),:]

print("Training set size:", len(train))
print("Test set size:", len(test))

# Visualize the original and normalized data
plt.figure(figsize=(10, 6))
plt.plot(data.index, data['Temp'], label='Original Data')
plt.title('Daily Minimum Temperatures in Melbourne')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
epochs = 100
batch_size = 32
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_lstm_model(input_shape, lstm_units=(50, 50), dropout_rate=0.2):
    model = Sequential()

    # Add the first LSTM layer
    model.add(LSTM(units=lstm_units[0], input_shape=input_shape, return_sequences=True))
    model.add(Dropout(rate=dropout_rate))

    # Add additional LSTM layers if specified
    if len(lstm_units) > 1:
        for units in lstm_units[1:]:
            model.add(LSTM(units=units, return_sequences=True))
            model.add(Dropout(rate=dropout_rate))

    # Add the output layer
    model.add(Dense(1))

    return model

# Define input shape (time steps, features)
X=df.iloc[:,:-1].values
input_shape = (X_train.shape[1], X_train.shape[2])

# Define LSTM units and dropout rate
lstm_units = (64, 64)  # Two LSTM layers with 64 units each
dropout_rate = 0.2

# Create the LSTM model
model = create_lstm_model(input_shape, lstm_units, dropout_rate)

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Print the model summary
model.summary()

In [ ]:
# Define number of epochs
epochs = 50

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=64, 
                    validation_data=(X_val, y_val), 
                    shuffle=True)

# Plot training/validation loss
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Define number of epochs
epochs = 50

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=64, 
                    validation_data=(X_val, y_val), 
                    shuffle=True)

# Plot training/validation loss
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Make predictions on the test set
test_predictions = model.predict(X_test)

# Invert predictions and actual values to original scale
test_predictions = scaler.inverse_transform(test_predictions)
y_test = scaler.inverse_transform([y_test])

# Calculate evaluation metrics
mae = mean_absolute_error(y_test[0], test_predictions[:, 0])
rmse = np.sqrt(mean_squared_error(y_test[0], test_predictions[:, 0]))
print('Mean Absolute Error (MAE):', mae)
print('Root Mean Squared Error (RMSE):', rmse)

# Visualize predictions vs actual values
plt.figure(figsize=(10, 6))
plt.plot(y_test[0], label='Actual')
plt.plot(test_predictions[:, 0], label='Predicted')
plt.title('Test Set Predictions vs Actual Values')
plt.xlabel('Time Steps')
plt.ylabel('Temperature')
plt.legend()
plt.show()